In [1]:
# Cell 1: Install Dependencies
!pip install -q transformers datasets peft accelerate bitsandbytes
!pip install nltk
!pip install fsspec==2025.3.0  # Match gcsfs requirement

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
from peft import PeftModel

In [3]:
from huggingface_hub import notebook_login
notebook_login()
#hf token

In [7]:
tokenizer = AutoTokenizer.from_pretrained("Aabe03/wb-lora-finetuned-model")
tokenizer.pad_token = tokenizer.eos_token


base_model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.2-3B",
    torch_dtype=torch.float16
)
base_model.to("cuda")

model = PeftModel.from_pretrained(base_model, "Aabe03/wb-lora-finetuned-model")
model = model.merge_and_unload()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_config.json:   0%|          | 0.00/795 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/48.7M [00:00<?, ?B/s]

In [19]:
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)

# Test
prompt = "This is a conference by Berkshire Hathaway and Warren Bufffett is answering questions, answer the question to the best of your ability as Warren Buffett may. Gentlemen, I’ll make this the last question. It comes from Phil King. He says, “Many people in the press and politics are questioning the validity of capitalism. What can you say to them that might prompt them to take a look at capitalism more favorably?”"
output = generator(
        prompt,
        max_length=500,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.1,
        num_return_sequences=1
    )
print("Generated text:", output[0]['generated_text'])

Device set to use cuda:0


Generated text: This is a conference by Berkshire Hathaway and Warren Bufffett is answering questions, answer the question to the best of your ability as Warren Buffett may. Gentlemen, I’ll make this the last question. It comes from Phil King. He says, “Many people in the press and politics are questioning the validity of capitalism. What can you say to them that might prompt them to take a look at capitalism more favorably?” Well, first of all, I think it’s very important to distinguish between capitalism and greed. Capitalism is not greed; it’s the opposite. It’s honesty and fairness and trying to do what’s right for everyone involved. And if you look at how our system has worked over many decades, it’s done quite well, even though there have been some bumps along the way. But we’ve had an incredibly productive period of time. We’ve had huge improvements in living standards. We’ve had innovations like computers, the Internet, cell phones, and so on. The standard of living has gone up